# Práctica 2b. 
Dada una imagen trazar una ventana de proyección y proyectar la imagen.

In [6]:
import cv2
import numpy as np

src_points = []
dst_points = []
phase = 0
image = None
clone = None
ruta_salida_global = None

def ordenar_cuatro_puntos(pts):
    pts = np.array(pts, dtype=np.float32)
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)
    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]
    tr = pts[np.argmin(diff)]
    bl = pts[np.argmax(diff)]
    return np.array([tl, tr, br, bl], dtype=np.float32)

def mouse_callback(event, x, y, flags, param):
    global src_points, dst_points, phase, image, clone
    if event == cv2.EVENT_LBUTTONDOWN:
        if phase == 0 and len(src_points) < 4:
            src_points.append([x, y])
            cv2.circle(image, (x, y), 5, (0, 255, 0), -1)
            cv2.imshow("Proyeccion interactiva", image)
            if len(src_points) == 4:
                print("Selecciona ahora los 4 puntos destino (rojo).")
                phase = 1
        elif phase == 1 and len(dst_points) < 4:
            dst_points.append([x, y])
            cv2.circle(image, (x, y), 5, (0, 0, 255), -1)
            cv2.imshow("Proyeccion interactiva", image)
            if len(dst_points) == 4:
                print("Calculando proyección...")
                proyectar()

def proyectar():
    global src_points, dst_points, clone, image, ruta_salida_global
    src = ordenar_cuatro_puntos(src_points)
    dst = ordenar_cuatro_puntos(dst_points)
    H, _ = cv2.findHomography(src, dst)
    h, w = clone.shape[:2]
    warped = cv2.warpPerspective(clone, H, (w, h))
    mask = np.zeros((h, w), dtype=np.uint8)
    cv2.fillPoly(mask, [np.int32(dst)], 255)
    result = np.zeros_like(clone)
    cv2.copyTo(warped, mask, result)
    cv2.imshow("Proyeccion interactiva", result)
    cv2.imwrite(ruta_salida_global, result)
    print(f"Resultado guardado en '{ruta_salida_global}'")

def main(ruta_entrada="images/cats.png", ruta_salida="output/resultado.png"):
    global image, clone, ruta_salida_global
    ruta_salida_global = ruta_salida
    clone = cv2.imread(ruta_entrada)
    if clone is None:
        raise FileNotFoundError(f"No se pudo cargar la imagen en {ruta_entrada}")
    image = clone.copy()
    cv2.namedWindow("Proyeccion interactiva")
    cv2.setMouseCallback("Proyeccion interactiva", mouse_callback)
    cv2.imshow("Proyeccion interactiva", image)
    print("Haz click en 4 puntos fuente (verde), luego en 4 puntos destino (rojo).")
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main("images/cats.png", "output/resultado.png")



Haz click en 4 puntos fuente (verde), luego en 4 puntos destino (rojo).
Selecciona ahora los 4 puntos destino (rojo).
Calculando proyección...
Resultado guardado en 'output/resultado.png'
